<a href="https://colab.research.google.com/github/DhanaLakshmiKotupalli/SQL-for-Data-Analysis-eCommerce-Dataset/blob/main/SQL_for_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
import pandas as pd

# Upload CSVs manually
uploaded = files.upload()

Saving customer.csv to customer.csv
Saving orderitem.csv to orderitem.csv
Saving orders.csv to orders.csv
Saving product.csv to product.csv


In [10]:
# Read CSVs into pandas DataFrames
df_customer = pd.read_csv("customer.csv")
df_product = pd.read_csv("product.csv")
df_orders = pd.read_csv("orders.csv")
df_orderitem = pd.read_csv("orderitem.csv")


In [11]:
import sqlite3

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Load DataFrames into the SQLite database
df_customer.to_sql("customer", conn, index=False, if_exists="replace")
df_product.to_sql("product", conn, index=False, if_exists="replace")
df_orders.to_sql("orders", conn, index=False, if_exists="replace")
df_orderitem.to_sql("orderitem", conn, index=False, if_exists="replace")



274566

In [13]:
# View a few rows
pd.read_sql("SELECT * FROM customer LIMIT 5", conn)


,customer_id,first_name,last_name,email,country
0,1,Brandon,Stevenson,brittany62@example.org,Saint Vincent and the Grenadines
1,2,Maria,Hendricks,burnsryan@example.com,Malaysia
2,3,Natalie,Hernandez,jonesmisty@example.org,Lao People's Democratic Republic
3,4,Amanda,Smith,timothyho@example.org,United Kingdom
4,5,Robin,Luna,rrivera@example.com,Martinique


##Top 5 Customers by Spending

In [19]:
query_top_customers = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    ROUND(SUM(oi.quantity * p.price), 2) AS total_spent
FROM customer c
JOIN orders o ON c.customer_id = o.customer_id
JOIN orderitem oi ON o.order_id = oi.order_id
JOIN product p ON oi.product_id = p.product_id
GROUP BY c.customer_id
ORDER BY total_spent DESC
LIMIT 5;
"""

pd.read_sql(query_top_customers, conn)


,customer_id,first_name,last_name,total_spent
0,429,Deanna,Harris,2184326.02
1,16,Carolyn,Russell,2117904.95
2,266,Eric,George,2108993.40
3,462,John,Robinson,2066853.04
4,337,Travis,May,2062195.97


##Top 5 Selling Products

In [22]:
query2 = """SELECT p.product_id, p.name,
       SUM(oi.quantity) AS total_units_sold
FROM product p
JOIN orderitem oi ON p.product_id = oi.product_id
GROUP BY p.product_id
ORDER BY total_units_sold DESC
LIMIT 5;
"""
pd.read_sql(query2, conn)


,product_id,name,total_units_sold
0,491,Luxury Watches - Product 1,3438
1,253,Office Furniture - Product 3,3395
2,38,Jeans - Product 3,3394
3,439,Monitors - Product 4,3369
4,221,Electronics - Product 1,3361


##Monthly Sales Trend

In [24]:
query_monthly_sales = """
SELECT
    strftime('%Y-%m', o.order_date) AS month,
    ROUND(SUM(oi.quantity * p.price), 2) AS monthly_sales
FROM orders o
JOIN orderitem oi ON o.order_id = oi.order_id
JOIN product p ON oi.product_id = p.product_id
GROUP BY month
ORDER BY month;
"""

pd.read_sql(query_monthly_sales, conn)


,month,monthly_sales
0,2016-01,9378167.65
1,2016-02,8537370.26
2,2016-03,10416868.58
3,2016-04,9354185.11
4,2016-05,9795990.05
...,...,...
79,2022-08,9805489.18
80,2022-09,9831306.89
81,2022-10,8548242.61
82,2022-11,9806533.69


##Create a View: Customer Order Summary

In [25]:
create_view_query = """
CREATE VIEW customer_order_summary AS
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(DISTINCT o.order_id) AS total_orders,
    ROUND(SUM(oi.quantity * p.price), 2) AS total_spent
FROM customer c
JOIN orders o ON c.customer_id = o.customer_id
JOIN orderitem oi ON o.order_id = oi.order_id
JOIN product p ON oi.product_id = p.product_id
GROUP BY c.customer_id;
"""

# Create the view
conn.execute(create_view_query)

# Preview the view
pd.read_sql("SELECT * FROM customer_order_summary LIMIT 5", conn)


,customer_id,first_name,last_name,total_orders,total_spent
0,1,Brandon,Stevenson,111,1592933.90
1,2,Maria,Hendricks,84,1359655.79
2,3,Natalie,Hernandez,92,1327424.50
3,4,Amanda,Smith,99,1537525.19
4,5,Robin,Luna,98,1625251.81


##Optimize with Index

In [26]:
conn.execute("CREATE INDEX idx_order_customer ON orders(customer_id)")


##Save Output as CSV

In [28]:
result = pd.read_sql(query_top_customers, conn)
result.to_csv("top_customers.csv", index=False)
files.download("top_customers.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>